In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.layers import LSTM, Dense 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2023-08-27 13:36:04.631033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
np.random.seed(0)

In [3]:
num_samples = 365
time_steps = 10

In [5]:
data_21 = np.random.randn(num_samples, time_steps)
df_21 = pd.DataFrame(data_21, columns=[f'feature_{i}' for i in range(time_steps)])
df_21['target'] = np.sin(np.arange(0, 36.5, 0.1)) + np.random.randn(num_samples) * 0.1
df_21.index = pd.date_range('2021-01-01', periods=num_samples)

In [6]:
scaler = MinMaxScaler()
scaled_data_21 = scaler.fit_transform(df_21.drop(columns='target'))
df_21.iloc[:, :-1] = scaled_data_21

In [7]:
X = df_21.drop(columns='target')
y = df_21['target']

In [8]:
X_3d = np.array([X.iloc[i:i+time_steps].values 
                 for i in range(len(X) - time_steps + 1)])
y_train = y[time_steps-1:]

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(time_steps, X.shape[1])),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(1)
])

2023-08-27 13:41:20.667876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-27 13:41:20.672238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-27 13:41:20.672400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(X_3d, y_train, epochs=50, batch_size=16, verbose=2)

Epoch 1/50


2023-08-27 13:41:38.845575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-08-27 13:41:39.024668: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0f54025b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-27 13:41:39.024694: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-08-27 13:41:39.028948: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-27 13:41:39.087220: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-27 13:41:39.135647: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23/23 - 3s - loss: 0.5171 - 3s/epoch - 149ms/step
Epoch 2/50
23/23 - 0s - loss: 0.5078 - 96ms/epoch - 4ms/step
Epoch 3/50
23/23 - 0s - loss: 0.5186 - 91ms/epoch - 4ms/step
Epoch 4/50
23/23 - 0s - loss: 0.5098 - 92ms/epoch - 4ms/step
Epoch 5/50
23/23 - 0s - loss: 0.5141 - 124ms/epoch - 5ms/step
Epoch 6/50
23/23 - 0s - loss: 0.4962 - 93ms/epoch - 4ms/step
Epoch 7/50
23/23 - 0s - loss: 0.4782 - 90ms/epoch - 4ms/step
Epoch 8/50
23/23 - 0s - loss: 0.4946 - 95ms/epoch - 4ms/step
Epoch 9/50
23/23 - 0s - loss: 0.4743 - 95ms/epoch - 4ms/step
Epoch 10/50
23/23 - 0s - loss: 0.4640 - 91ms/epoch - 4ms/step
Epoch 11/50
23/23 - 0s - loss: 0.4580 - 90ms/epoch - 4ms/step
Epoch 12/50
23/23 - 0s - loss: 0.4526 - 97ms/epoch - 4ms/step
Epoch 13/50
23/23 - 0s - loss: 0.4427 - 95ms/epoch - 4ms/step
Epoch 14/50
23/23 - 0s - loss: 0.4311 - 93ms/epoch - 4ms/step
Epoch 15/50
23/23 - 0s - loss: 0.4439 - 96ms/epoch - 4ms/step
Epoch 16/50
23/23 - 0s - loss: 0.4440 - 98ms/epoch - 4ms/step
Epoch 17/50
23/23 - 0s - lo

In [11]:
num_future_days = 31
future_data = np.random.randn(num_future_days, time_steps)
future_df = pd.DataFrame(future_data, columns=[f'feature_{i}' for i in range(time_steps)])
future_df.index = pd.date_range('2022-01-01', periods=num_future_days)

In [12]:
scaled_future_data = scaler.transform(future_df)
future_df.iloc[:, :] = scaled_future_data

In [14]:
future_df = pd.concat([X.tail(time_steps-1), future_df])

In [17]:
# 입력 데이터 형식 변환 (3D 배열)
X_future_3d = np.array([future_df.iloc[i:i+time_steps].values for i in range(len(future_df) - time_steps + 1)])

# 모델을 사용하여 22년 1월의 타겟 예측
predicted_targets = model.predict(X_future_3d)

# 예측 결과를 역정규화하여 실제 값으로 변환
predicted_targets = scaler.inverse_transform(np.concatenate([X_future_3d[:, -1, :-1], predicted_targets], axis=1))[:, -1]

# 예측 결과 출력
predicted_df = pd.DataFrame(predicted_targets, columns=['predicted_target'], index=future_df.index[(time_steps-1):])
print(predicted_df)

1/1 [==============================] - 0s 19ms/step


            predicted_target
2022-01-01         -1.249488
2022-01-02         -1.595428
2022-01-03          0.077462
2022-01-04          0.674467
2022-01-05         -0.777405
2022-01-06         -1.635720
2022-01-07         -0.307455
2022-01-08          1.153783
2022-01-09          0.638511
2022-01-10          0.907252
2022-01-11          1.203158
2022-01-12          1.083635
2022-01-13          1.160434
2022-01-14          0.982527
2022-01-15          1.264416
2022-01-16          1.952493
2022-01-17          1.470261
2022-01-18         -0.067756
2022-01-19         -0.732403
2022-01-20         -2.000589
2022-01-21         -0.959475
2022-01-22          0.424397
2022-01-23          1.062350
2022-01-24         -0.366932
2022-01-25         -1.292703
2022-01-26         -1.911557
2022-01-27         -2.603988
2022-01-28         -0.081385
2022-01-29          1.006905
2022-01-30          1.427867
2022-01-31          2.223300


: 